# Experimental Results for Paper

In [2]:
import dp4gp_datasets
import dp4gp
import random
import numpy as np
import GPy
import matplotlib.pyplot as plt
import dp4gp_histogram
import pandas as pd
%matplotlib inline

## House prices

In [3]:
def get_house_prices():
    #Skip this if you want, and load precomputed data from the csv file below:
    #dp4gp_datasets.prepare_preloaded_prices('test.csv', boundingbox=[480e3, 130e3, 580e3, 230e3], N=10000, col_list=['QS501EW'])
    #Load precomputed dataset:
    dataset = pd.read_csv('price_dataset10k.csv') #london, 10k purchases

    #Reduce the size of the dataset and split into training and test data:
    random.seed(123)
    #dataset = dataset[dataset['years']>2005]
    dataset = dataset.ix[random.sample(dataset.index, 200)]

    #get into useful form
    #east, north, time, education -> price
    inputs = np.vstack([dataset['easting'].values,dataset['northing'].values,dataset['seconds'].values,(dataset['QS501EW_6']/dataset['QS501EW_0']).values]).T

    #inputs = inputs[:,0:2]
    ys = dataset['price'].values
    
    return inputs, ys

In [4]:
def get_citibike_data():
    allcb = dp4gp_datasets.load_citibike(station=None)
    subcb = allcb[allcb['usertype']=='Subscriber']
    random.seed(123)

    cb = subcb.ix[random.sample(subcb.index, 5000)]
    inputs = np.c_[cb['start station latitude'],cb['end station latitude'],cb['start station longitude'],cb['end station longitude']]
    ys = cb['tripduration'].values
    
    return inputs, ys

In [5]:

def get_noDP_prediction(training_inputs, training_ys, test_inputs, sens, eps, delta,noise,modvar,kernval,kern_ls,steps):
    rbf = GPy.kern.RBF(training_inputs.shape[1],kernvar, kern_ls,ARD=True)
    mod = GPy.models.GPRegression(training_inputs,training_ys,rbf)
    mod.Gaussian_noise = noise
    dpgp = dp4gp.DPGP_cloaking(mod,sens,eps,delta)
    preds, mu, cov = dpgp.draw_prediction_samples(test_inputs,1,1,0)
    return mu, None


def get_cloaking_prediction(training_inputs, training_ys, test_inputs, sens, eps, delta,noise,modvar,kernval,kern_ls,steps):
    rbf = GPy.kern.RBF(training_inputs.shape[1],kernvar, kern_ls,ARD=True)
    #rbf = GPy.kern.RBF(training_inputs.shape[1],modvar,[5e3,5e3],ARD=True)
    mod = GPy.models.GPRegression(training_inputs,training_ys,rbf)
    mod.Gaussian_noise = noise
    dpgp = dp4gp.DPGP_cloaking(mod,sens,eps,delta)
    preds, mu, cov = dpgp.draw_prediction_samples(test_inputs,1,1,1000)
    return preds, cov
    
def get_integral_prediction(training_inputs, training_ys, test_inputs, sens, eps, delta,noise,modvar,kernval,kern_ls,steps):
    Xtest, free_inputs, step = dp4gp.compute_Xtest(training_inputs,steps=steps)
    print step
    dpgp = dp4gp_histogram.DPGP_histogram(sens,eps,delta)
    dpgp.prepare_model(Xtest,training_inputs,step,training_ys,lengthscale=kern_ls)
    #dpgp.optimize()
    dpgp.model.optimize(messages=True)
    preds, cov = dpgp.draw_prediction_samples(test_inputs)
    return preds, cov

def get_standard_prediction(training_inputs, training_ys, test_inputs, sens, eps, delta,noise,modvar,kernval,kern_ls,steps):
    rbf = GPy.kern.RBF(training_inputs.shape[1],kernvar,kern_ls,ARD=True)
    mod = GPy.models.GPRegression(training_inputs,training_ys,rbf)
    mod.Gaussian_noise = noise
    dpgp = dp4gp.DPGP_normal_prior(mod,sens,eps,delta)
    preds, mu, cov = dpgp.draw_prediction_samples(test_inputs,1)
    return preds, cov

def get_pseudo_prediction(training_inputs, training_ys, test_inputs, sens, eps, delta,noise,modvar,kernval,kern_ls,steps):
    rbf = GPy.kern.RBF(training_inputs.shape[1],kernvar,kern_ls,ARD=True)
    print training_ys.ndim
    print training_inputs.shape
    mod = GPy.models.SparseGPRegression(training_inputs,training_ys,kernel=rbf,num_inducing=40) #no idea how many inducing!
    mod.inference_method = GPy.inference.latent_function_inference.FITC()
    mod.set_Z(training_inputs[0:40,:]) #grab random inputs as pseudoinputs
    mod.Gaussian_noise = noise  
    dpgp = dp4gp.DPGP_pseudo_prior(mod,sens,eps,delta)
    preds, mu, cov = dpgp.draw_prediction_samples(test_inputs,1)
    return preds, cov

In [18]:
#inputs, ys = get_house_prices()
inputs, ys = get_citibike_data()

#FOR CITIBIKE
#squash data into 0-2000 seconds range
ys[ys>2000] = 2000
ys[ys<0] = 0
sens = 2000-0
kernvar = 10.0
kern_ls = np.array([0.01,0.01,0.01,0.01])*5.0

#FOR HOUSE PRICES
#squash data into £10k-£500k range
#ys[ys>5e5] = 5e5
#ys[ys<1e4] = 1e4
#sens = 5e5-1e4
#kernvar = 1.0
#kern_ls = [15e3,15e3,50*31536000,5.0]

ys_mean = np.mean(ys)
ys_std = np.std(ys)
ys = ys - ys_mean
ys = ys / ys_std
sens = sens / ys_std

training_inputs = inputs[0:-100,:]
training_ys = ys[0:-100][:,None]
test_inputs = inputs[-100:,:]
test_ys = ys[-100:][:,None]

fns = [get_noDP_prediction, get_integral_prediction, get_cloaking_prediction]#,get_pseudo_prediction,get_standard_prediction]
labels = ["No DP","Integral","Cloaking"]   #"Pseudo","Standard"]

for steps in [1,3,6,10]:
    for eps in [0.01,0.1,0.2,0.5,1.0]:
        results = []
        for fn,label in zip(fns,labels):
            preds, cov = fn(training_inputs,training_ys,test_inputs,sens,eps,0.01,10.0,1.0, kernvar, kern_ls, steps)
            RMSE = np.sqrt(np.mean((test_ys-preds)**2))
            results.append({'label':label, 'preds':preds, 'cov':cov, 'RMSE':RMSE})
        print "EPSILON = %0.3f" % eps
        print "STEPS = %0.3f" % steps
        for r in results:
            print r['label'],r['RMSE']

(100, 4900)
*
Stopped before convergence
(3.9836146099035994, 3.2552472614374586, array([[ 1.26196074]]), 0.01, 0.0)
[ 0.12933827  0.12996228  0.10469202  0.15097224]
(100, 4900)
*
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

MemoryError: 

In [15]:
for r in results:
    print r['label'],r['RMSE']#*ys_std

#5000 users
#NO DIFFERENTIAL PRIVACY
# LS,  VAR    RMSE
#0.05  100 ->  -
#0.05    1 -> 0.76
#0.05   10 -> 0.67
#0.02   10 -> 0.64
#0.01, 100 -> 0.65
#0.01,  10 -> 0.66
#0.005,100 -> 0.78

#LS=0.05, VAR=10, EPSILON=1
#NO DP            0.6755
#INTEGRAL(step=1) 0.9755 [not enough iterations allowed] !! ONE BIN !!
#INTEGRAL(step=4) 1.3314
#INTEGRAL(step=6) 1.3981
#CLOAK            0.7539




No DP 0.675569194489
Integral 1.39811547635


DATA FOR HOUSE PRICES - DELETE ALL THIS !!! OLD BROKEN!!!

    Number of data: 1000 points
    (training_inputs,training_ys[:,None],test_inputs,sens,1.0,0.01,2.0,1.0)
    .RBF(training_inputs.shape[1],1.0,[15e3,15e3,50*31536000,5.0],ARD=True)
    integral grid steps = 10 (1000 points!)
    nits = 100

        No DP 1.07821272291
        Integral 0.995404374762
        Cloaking 1.56684865485

    (training_inputs.shape[1],1.0,[15e3,15e3,50*31536000,5.0],ARD=True)
    .RBF(training_inputs.shape[1],1.0,[25e3,25e3,50*31536000,5.0],ARD=True)
    (Xtest,training_inputs,step,ys,lengthscale=[25e3,25e3])
        No DP 1.07821272291
        Integral 1.07568810881
        Cloaking 1.39788247853





    Noise = 5.0, lengthscale = 5km
    (training_inputs,training_ys[:,None],test_inputs,sens,1.0,0.01,5.0,1.0)
    ===500 points (400 training, 100 test)===
    2d
    Cloaking 1.97518575121
    Pseudo 150.294285029
    Standard 3.30680222508

    4d
    Cloaking 2.70756061351
    Pseudo 81.3098354009
    Standard 2.81548412873
    

    ===5000 points (4900 training, 100 test)===

    4d
    Cloaking 1.12417092118
    Pseudo 653.231209888
    Standard 5.89137411916
    
    More lengthscale;
    Noise = 5.0, lengthscale = 15km
    (training_inputs.shape[1],1.0,[15e3,15e3,50*31536000,5.0],ARD=True)
    ===5000 points (4900 training, 100 test)===
    
    4d
    Cloaking 1.13735358897
    Pseudo 6653.4975417
    Standard 6.43561788669

    Less noise;
    Noise = 2.0, lengthscale = 15km
    (training_inputs.shape[1],1.0,[15e3,15e3,50*31536000,5.0],ARD=True)
    ===5000 points (4900 training, 100 test)===
    4d
    Cloaking 1.17823238187
    Pseudo 12793.3710843
    Standard 19.4062574837

    ===10k points===
    4d
    Cloaking 1.2916
    Pseudo 82754.88
    
    It looks like more points doesn't help over ~1k   
    
    fn(training_inputs,training_ys[:,None],test_inputs,sens,1.0,0.01,2.0,1.0)
    (training_inputs.shape[1],1.0,[15e3,15e3,50*31536000,5.0],ARD=True)
    ===1000 points (900 training, 100 test)===
    4d
    Cloaking 1.51661914526
    Pseudo 935.593475553
    Standard 17.311877784

### Other code

In [ ]:
plt.figure(figsize=[14,10])
for i,r in enumerate(results):
    
    plt.subplot(2,2,i+1)
    plt.title(r['label'])
    plt.errorbar(test_ys,r['preds'],1.96*np.sqrt(np.diag(r['cov'])),fmt=None,color='k',alpha=0.2)
    plt.plot(test_ys,r['preds'],'+k')
    #plt.xlim([0,1e6])
    #plt.ylim([0,1e6])
    #plt.plot([0,1e6],[0,1e6],'k-')
    plt.xlim([-2,4])
    plt.ylim([-2,4])
    plt.plot([-2,4],[-2,4],'k-')


In [ ]:
def plot_cloaking_prediction(training_inputs, training_ys, test_inputs, sens, eps, delta,noise,modvar):
    rbf = GPy.kern.RBF(training_inputs.shape[1],1.0,[5e3,5e3,50*31536000,5.0],ARD=True)
    #rbf = GPy.kern.RBF(training_inputs.shape[1],modvar,[5e3,5e3],ARD=True)
    mod = GPy.models.GPRegression(training_inputs,training_ys,rbf)
    mod.Gaussian_noise = noise
    dpgp = dp4gp.DPGP_cloaking(mod,sens,eps,delta)
    return dpgp
dpgp = plot_cloaking_prediction(training_inputs,training_ys[:,None],test_inputs,sens,1.0,0.01,5.0,1.0)

In [ ]:
dpgp.plot(fixed_inputs=[(2,np.mean(training_inputs[:,2])),(3,np.mean(training_inputs[:,3]))])

In [ ]:
np.mgrid[0:2]

In [ ]:
np.mgrid[0:2,0:3]

In [ ]:
np.mgrid[0:2,0:3,0:4][0].flatten()